In [4]:
import gc
import pickle
import gensim
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler


In [13]:
click_log=pd.read_pickle('/Users/hesu/Documents/KT/Rank1st-master/data/click.pkl')
train_df=pd.read_pickle('/Users/hesu/Documents/KT/Rank1st-master/data/train_user.pkl')
test_df=pd.read_pickle('/Users/hesu/Documents/KT/Rank1st-master/data/test_user.pkl')
print(click_log.shape,train_df.shape,test_df.shape)

(33586512, 23) (900000, 3) (38, 3)


In [14]:
click_log.head(10)

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,age,...,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,gender_0,gender_1
0,1,3805796,134762,1,121137,42310,17,25032,-1,-1,...,0,0,0,0,0,0,0,0,0,0
1,1,3508375,258797,1,232819,2563,13,33689,247,-1,...,0,0,0,0,0,0,0,0,0,0
2,1,3067846,209780,1,188510,5912,3,22806,322,-1,...,0,0,0,0,0,0,0,0,0,0
3,1,3735729,185172,1,166034,26858,3,2092,60,-1,...,0,0,0,0,0,0,0,0,0,0
4,1,3735729,232037,1,208440,5053,3,2384,247,-1,...,0,0,0,0,0,0,0,0,0,0
5,1,3376816,185172,1,166034,26858,3,2092,60,-1,...,0,0,0,0,0,0,0,0,0,0
6,1,3005643,40025,1,38334,1447,2,24691,238,-1,...,0,0,0,0,0,0,0,0,0,0
7,1,3377203,84174,1,77112,12178,8,32989,232,-1,...,0,0,0,0,0,0,0,0,0,0
8,1,3005643,116453,1,105247,1531,2,22965,319,-1,...,0,0,0,0,0,0,0,0,0,0
9,1,3735729,177742,1,159402,2571,2,16326,247,-1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
click_log.columns

Index(['time', 'user_id', 'creative_id', 'click_times', 'ad_id', 'product_id',
       'product_category', 'advertiser_id', 'industry', 'age', 'gender',
       'age_0', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7',
       'age_8', 'age_9', 'gender_0', 'gender_1'],
      dtype='object')

In [16]:
train_df.head(10)

,user_id,age,gender
0,1,3,0
1,2,9,0
2,3,6,1
3,4,4,0
4,5,3,0
5,6,5,0
6,7,5,1
7,8,4,0
8,9,4,0
9,10,8,1


In [17]:
test_df.head(10)

,user_id,age,gender
0,3131989,-1,-1
1,3131990,-1,-1
2,3131994,-1,-1
3,3131995,-1,-1
4,3132000,-1,-1
5,3132004,-1,-1
6,3132007,-1,-1
7,3132009,-1,-1
8,3132011,-1,-1
9,3132012,-1,-1


# 聚合特征

In [8]:
def get_agg_features(dfs,f1,f2,agg,log):
    # [train_df,test_df],'user_id','','size',click_log
    # [train_df,test_df],'user_id','creative_id','unique',click_log
    # 判定特殊情况
    if type(f1)==str:
        f1=[f1]
    if agg!='size':
        data=log[f1+[f2]]
    else:
        data=log[f1] 
    f_name='_'.join(f1)+"_"+f2+"_"+agg     
    # 聚合操作
    if agg=="size":
        tmp = pd.DataFrame(data.groupby(f1).size()).reset_index()
    elif agg=="count":
        tmp = pd.DataFrame(data.groupby(f1)[f2].count()).reset_index()
    elif agg=="mean":
        tmp = pd.DataFrame(data.groupby(f1)[f2].mean()).reset_index()
    elif agg=="unique":
        tmp = pd.DataFrame(data.groupby(f1)[f2].nunique()).reset_index()
    elif agg=="max":
        tmp = pd.DataFrame(data.groupby(f1)[f2].max()).reset_index()
    elif agg=="min":
        tmp = pd.DataFrame(data.groupby(f1)[f2].min()).reset_index()
    elif agg=="sum":
        tmp = pd.DataFrame(data.groupby(f1)[f2].sum()).reset_index()
    elif agg=="std":
        tmp = pd.DataFrame(data.groupby(f1)[f2].std()).reset_index()
    elif agg=="median":
        tmp = pd.DataFrame(data.groupby(f1)[f2].median()).reset_index()
    else:
        raise "agg error"   
    # 赋值聚合特征
    for df in dfs:
        try:
            del df[f_name]
        except:
            pass
        tmp.columns = f1+[f_name]
        df[f_name]=df.merge(tmp, on=f1, how='left')[f_name] 
    del tmp
    del data
    gc.collect()
    return [f_name]

In [18]:
agg_features=[]
agg_features+=get_agg_features([train_df,test_df],'user_id','','size',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','ad_id','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','creative_id','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','advertiser_id','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','industry','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','product_id','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','time','unique',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','click_times','sum',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','click_times','mean',click_log)
agg_features+=get_agg_features([train_df,test_df],'user_id','click_times','std',click_log)

In [19]:
agg_features
# 这个agg_features里面存储的是一个个的特征的name，对应的特征值，都加入到了train_df和test_df里面去了
# 统计click里面的一些user等的信息，再存储到train_df和test_df里面去

['user_id__size',
 'user_id_ad_id_unique',
 'user_id_creative_id_unique',
 'user_id_advertiser_id_unique',
 'user_id_industry_unique',
 'user_id_product_id_unique',
 'user_id_time_unique',
 'user_id_click_times_sum',
 'user_id_click_times_mean',
 'user_id_click_times_std']

In [20]:
click_log.head(10)

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,age,...,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,gender_0,gender_1
0,1,3805796,134762,1,121137,42310,17,25032,-1,-1,...,0,0,0,0,0,0,0,0,0,0
1,1,3508375,258797,1,232819,2563,13,33689,247,-1,...,0,0,0,0,0,0,0,0,0,0
2,1,3067846,209780,1,188510,5912,3,22806,322,-1,...,0,0,0,0,0,0,0,0,0,0
3,1,3735729,185172,1,166034,26858,3,2092,60,-1,...,0,0,0,0,0,0,0,0,0,0
4,1,3735729,232037,1,208440,5053,3,2384,247,-1,...,0,0,0,0,0,0,0,0,0,0
5,1,3376816,185172,1,166034,26858,3,2092,60,-1,...,0,0,0,0,0,0,0,0,0,0
6,1,3005643,40025,1,38334,1447,2,24691,238,-1,...,0,0,0,0,0,0,0,0,0,0
7,1,3377203,84174,1,77112,12178,8,32989,232,-1,...,0,0,0,0,0,0,0,0,0,0
8,1,3005643,116453,1,105247,1531,2,22965,319,-1,...,0,0,0,0,0,0,0,0,0,0
9,1,3735729,177742,1,159402,2571,2,16326,247,-1,...,0,0,0,0,0,0,0,0,0,0


In [21]:
click_log.columns

Index(['time', 'user_id', 'creative_id', 'click_times', 'ad_id', 'product_id',
       'product_category', 'advertiser_id', 'industry', 'age', 'gender',
       'age_0', 'age_1', 'age_2', 'age_3', 'age_4', 'age_5', 'age_6', 'age_7',
       'age_8', 'age_9', 'gender_0', 'gender_1'],
      dtype='object')

In [22]:
train_df.head(10)

,user_id,age,gender,user_id__size,user_id_ad_id_unique,user_id_creative_id_unique,user_id_advertiser_id_unique,user_id_industry_unique,user_id_product_id_unique,user_id_time_unique,user_id_click_times_sum,user_id_click_times_mean,user_id_click_times_std
0,1,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,9,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,5,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
test_df.head(10)

,user_id,age,gender,user_id__size,user_id_ad_id_unique,user_id_creative_id_unique,user_id_advertiser_id_unique,user_id_industry_unique,user_id_product_id_unique,user_id_time_unique,user_id_click_times_sum,user_id_click_times_mean,user_id_click_times_std
0,3131989,-1,-1,29,25,25,19,11,10,13,29,1.000000,0.000000
1,3131990,-1,-1,41,36,37,29,12,8,20,50,1.219512,0.524985
2,3131994,-1,-1,13,13,13,13,5,12,10,15,1.153846,0.375534
3,3131995,-1,-1,17,14,14,14,7,9,14,19,1.117647,0.332106
4,3132000,-1,-1,18,17,17,15,12,6,14,19,1.055556,0.235702
5,3132004,-1,-1,14,14,14,14,10,10,10,15,1.071429,0.267261
6,3132007,-1,-1,41,39,39,23,23,13,27,42,1.024390,0.156174
7,3132009,-1,-1,97,94,94,75,30,45,54,101,1.041237,0.199871
8,3132011,-1,-1,48,46,46,39,14,23,24,49,1.020833,0.144338
9,3132012,-1,-1,78,74,74,63,26,32,44,80,1.025641,0.159085


In [24]:
train_df.columns

Index(['user_id', 'age', 'gender', 'user_id__size', 'user_id_ad_id_unique',
       'user_id_creative_id_unique', 'user_id_advertiser_id_unique',
       'user_id_industry_unique', 'user_id_product_id_unique',
       'user_id_time_unique', 'user_id_click_times_sum',
       'user_id_click_times_mean', 'user_id_click_times_std'],
      dtype='object')

In [25]:
test_df.columns

Index(['user_id', 'age', 'gender', 'user_id__size', 'user_id_ad_id_unique',
       'user_id_creative_id_unique', 'user_id_advertiser_id_unique',
       'user_id_industry_unique', 'user_id_product_id_unique',
       'user_id_time_unique', 'user_id_click_times_sum',
       'user_id_click_times_mean', 'user_id_click_times_std'],
      dtype='object')

In [26]:
def sequence_text(dfs,f1,f2,log):
    f_name='sequence_text_'+f1+'_'+f2
    print(f_name)
    # 遍历log，获得用户的点击序列
    dic,items={},[]
    for item in log[[f1,f2]].values:
        try:
            dic[item[0]].append(str(item[1]))
        except:
            dic[item[0]]=[str(item[1])]      
    for key in dic:
        items.append([key,' '.join(dic[key])])
    # 赋值序列特征
    temp=pd.DataFrame(items)
    temp.columns=[f1,f_name]
    temp = temp.drop_duplicates(f1)
    for df in dfs:
        try:
            del df[f_name]
        except:
            pass
        temp.columns = [f1]+[f_name]
        df[f_name]=df.merge(temp, on=f1, how='left')[f_name]
    gc.collect() 
    del temp
    del items
    del dic
    return [f_name]

In [ ]:
text_features=[]
text_features+=sequence_text([train_df,test_df],'user_id','ad_id',click_log)
